# McWhorter Problem
<cite>[McWhorter and Sunada][1]</cite> propose an analytical solution to the two-phase flow equation. A one-dimensional problem was considered which describes the flow of two incompressible, immiscible fluids through a porous medium, where the wetting phase (water) displaces the non-wetting fluid (air or oil) in the horizontal direction (without the influence of gravity).

![mcWhorter_concept_schematic.png](figures/mcWhorter_concept.png)

### Material Parameters

| Property  | Symbol | Value | Unit |
|----------:|:------:|-------|------|
| Porosity | $\phi$ | $$0.15$$ | 1 |
| Intrinsic permeability | $K$ | $$1.0\cdot 10^{-10}$$ | $m^2$|
| Residual saturation of the wetting phase | $$s_\mathrm{L}^{res}$$ | $$0.02$$ | 1 |
| Residual saturation of the non-wetting phase | $$s_\mathrm{G}^{res}$$ | $$0.001$$ | 1 |
| Dynamic viscosity of the wetting phase	|$\mu_\mathrm{L}$|$$1.0\cdot 10^{-3}$$|Pa s|
| Dynamic viscosity of the non-wetting pha  |$\mu_\mathrm{G}$  |$$5.0\cdot 10^{-3}$$|Pa s|
| Brooks and Corey model parameter: entry pressure | $p_b$ | $$5000$$| Pa |
| Brooks and Corey model parameter: pore size distribution index |$\lambda$ | $$3.0$$ | 1 |

### Problem Parameters

| Property  | Symbol | Value | Unit |
|----------:|:------:|-------|------|
| Initial saturation | $$s_\mathrm{L}(t=0)$$ | 0.05 | 1 |
| Injection boundary saturation | $$s_\mathrm{L}(x=0)$$ | 0.8 | 1 |


[1]: https://doi.org/10.1029/WR026i003p00399

In [3]:
param = {"mun": 1.e-3,
         "muw": 1.e-3,
         "K": 1e-10,
         "phi": 0.3,
         "lambda": 2,
         "Pe": 5000,
         "pc0": 5000.0,
         "pci": 5.e4}

## Analytical solution
The analytical solution is calculated according to <cite>[McWhorter and Sunada][1]</cite>.

[1]: https://doi.org/10.1029/WR026i003p00399

In [ ]:
from mcworther import BrooksCorey, McWorther

model = BrooksCorey(pd=param["Pe"], lambda_=param["lambda"])

pc_0 = param["pc0"]
pc_i = param["pci"]

S0 = model.Sw(pc_0)
Si = model.Sw(pc_i)

print(f"S0 = {S0}, Si = {Si:.3f}")

problem = McWorther(model, 
                    phi=param["phi"],
                    K=param["K"],
                    muw=param["muw"],
                    mun=param["mun"],
                    S0=S0,
                    Si=Si)

t_ref, Sw_ref = problem.get_solution()

## Numerical solutions

For the numerical solution, we compare the Thermal-2-Phase-Hydro-Mechanical (TH2M) and the Two-phase Flow formulation.

In [ ]:
import ogstools as ogs
from template import prj_from_template

# Simulate problem with TwoPhaseFlowPP and TH2M

template_files = [r"TwoPhase_mcwt_line.template", r"mcWorther_h2.template"]
prj_files = [r"TwoPhase_mcwt_line_test.prj", r"mcWorther_h2_test.prj"]

for template_file, prj_file in zip(template_files, prj_files):
    prj_from_template(param, template_file, prj_file)
    prj = ogs.Project(input_file=prj_file, output_file=prj_file)
    prj.run_model()

## Evaluation and Results

In [ ]:
import numpy as np
import pyvista as pv
from helper import plot_with_error


# 1. Plot analytical solution
plotter = plot_with_error(t_ref, Sw_ref, "C0-",
                          "Analytical solution",
                          "Distance $x$ (m)",
                          "Wetting saturation $S_w$ (-)",
                          "Error (Numerical - Analytical) (-)")

# 2. Read results from TwoPhaseFlowPP

labels = ["TwoPhaseFlowPP", "TH2M"]
results = ["twophaseflow_test_t_1000.000000.vtu", "mcWhorter_h2_newton_ts_110_t_1000.000000.vtu"]


i = 1
for label, result in zip(labels, results):

    mesh = pv.read(result)
    print(mesh.point_data.keys())

    Sw=mesh["saturation"]
    x=mesh.points[:,0]

    ind = np.argsort(x)
    x = x[ind]
    Sw = Sw[ind]

    plotter.append(x, Sw, f"C{i}x", label)
    i = i + 1 


# show plot

plotter.plot([0, 0.5])

## Literature

McWhorter, D. B., and D. K. Sunada (1990), Exact integral solutions for two-phase flow, Water Resour. Res., 26(3), 399–413, <cite>[doi:10.1029/WR026i003p00399][1]</cite>. 

[1]: https://doi.org/10.1029/WR026i003p00399
